### Introduction to Tools in LangChain

Large Language Models (LLMs) are powerful reasoning engines, but they are isolated from the outside world. They can't access real-time data or perform actions on their own. **Tools** bridge this gap.

Tools allow models to interact with the world, such as:
-   Fetching data from a database or API (e.g., weather, stock prices).
-   Searching the web.
-   Performing complex calculations.

**Core Concepts of a Tool:**
1.  **Schema**: A structured definition (JSON) that tells the model the tool's **name**, **description**, and **arguments** (inputs). The model uses this to "decide" if and how to call the tool.
2.  **Function**: The actual Python code that performs the logic (the "work").
3.  **Tool Object**: A wrapper in LangChain that combines the function and schema.

### 1. Defining a Tool with `@tool`

The `@tool` decorator is the easiest way to define a custom tool. It automatically extracts the schema from your Python function:
-   **Name**: Taken from the function name (e.g., `get_weather`).
-   **Description**: Taken from the function's docstring. **Crucial**: This tells the model *when* to use the tool.
-   **Arguments**: Taken from the function arguments and type hints (e.g., `location: str`).

In [20]:

import os
from langchain.chat_models import init_chat_model

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

model = init_chat_model("groq:qwen/qwen3-32b")
response = model.invoke("Why do parrots talk?")
response

AIMessage(content='<think>\nOkay, so I need to figure out why parrots talk. Let me start by recalling what I know about parrots. They\'re birds, right? Some species like the African Grey or the Amazon parrot are known for their ability to mimic human speech. But why do they do that? Is it just a learned behavior, or is there an evolutionary reason behind it?\n\nFirst, I remember that birds have a syrinx, which is a vocal organ that allows them to produce a range of sounds. But humans have a larynx. So parrots must have some unique structure or ability that enables them to mimic sounds. Maybe they use their syrinx in a specific way to replicate human speech. But why would they evolve that ability?\n\nI think communication is a big part of animal behavior. Parrots are social animals, so maybe talking helps them interact with each other. In the wild, maybe they mimic sounds to communicate with flock members, like alarm calls or food calls. If that\'s the case, then talking in captivity co

### 2. Binding Tools to the Model

Start by initializing a chat model. Then, use `.bind_tools([tool_list])`.

**Why bind tools?**
Standard LLMs generate text. Models fine-tuned for tool calling (like `llama-3.1-8b`, `gpt-4`, etc.) need to know what tools are available. `bind_tools` converts your Python tool definitions into the specific JSON format the model provider expects (e.g., OpenAI function calling schema).

In [22]:
from langchain.tools import tool

@tool
def get_weather(location:str)->str:
    """Get the weather at a location"""
    return f"It's sunny in {location}"


model_with_tools=model.bind_tools([get_weather])


response = model_with_tools.invoke("What's the weather like in Boston?")
print(response)
for tool_call in response.tool_calls:
    # View tool calls made by the model
    print(f"Tool: {tool_call['name']}")
    print(f"Args: {tool_call['args']}")

content='' additional_kwargs={'reasoning_content': "Okay, the user is asking about the weather in Boston. I need to use the get_weather function. Let me check the function parameters. It requires a location, which in this case is Boston. I'll call the function with Boston as the argument. Make sure the JSON is correctly formatted with the name and arguments. No other tools are available, so this should be straightforward.\n", 'tool_calls': [{'id': 'scvvsjxjq', 'function': {'arguments': '{"location":"Boston"}', 'name': 'get_weather'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 100, 'prompt_tokens': 154, 'total_tokens': 254, 'completion_time': 0.180425961, 'completion_tokens_details': {'reasoning_tokens': 76}, 'prompt_time': 0.008336504, 'prompt_tokens_details': None, 'queue_time': 3.096976921, 'total_time': 0.188762465}, 'model_name': 'qwen/qwen3-32b', 'system_fingerprint': 'fp_d58dbe76cd', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'l

### 3. The Agent Loop (Manual Implementation)

Modern Agents (like those in **LangGraph**) typically run a loop. Here, we implement a single iteration manually to understand what happens under the hood.

**The Workflow:**

1.  **Thinking (Model Generation)**: We send the user's query to the model. The model analyzes the query and the available tools.
2.  **Tool Call Generation**: Instead of a text response, the model returns a **Tool Call**. This contains:
    -   The name of the tool to call (`get_weather`).
    -   The arguments to pass (`{"location": "Kolkata"}`).
    -   A unique ID for the call.
3.  **Execution**: The application (us) detects the tool call. We invoke the actual Python function `get_weather` using the provided arguments.
4.  **Observation**: We get the output string ("It is sunny in Kolkata."). We must pass this back to the model as a `ToolMessage` linked to the original call ID.
5.  **Final Response**: The model receives the tool's output. It uses this information to generate a natural language response for the user.

In [23]:
# Step 1: Model generates tool calls
messages = [{"role": "user", "content": "What's the weather in Boston?"}]
ai_msg = model_with_tools.invoke(messages)
messages.append(ai_msg)

# Step 2: Execute tools and collect results
for tool_call in ai_msg.tool_calls:
    # Execute the tool with the generated arguments
    tool_result = get_weather.invoke(tool_call)
    messages.append(tool_result)

# Step 3: Pass results back to model for final response
final_response = model_with_tools.invoke(messages)
print(final_response.text)
# "The current weather in Boston is 72°F and sunny."

The weather in Boston is sunny right now. It's a great day to enjoy outdoor activities!
